In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import random
from sklearn.model_selection import train_test_split
import pickle

In [2]:
### Constants
DATA_PATH = "./train_data" # path to the dataset
LABEL_PATH = './media/labels.csv' # path to the labels csv file
RESULT_PATH = "./model_trained.p" # path to the labels csv file
BATCH_SIZE = 50  # size of batches during training
IMG_DIM = (32, 32)  # dimensions of the images (height, width)
TEST_SPLIT = 0.2  # proportion of data used for testing
VAL_SPLIT = 0.2  # proportion of training data for validation
N_EPOCHS = 30  # number of epochs for training

In [3]:
### Custom Dataset
class TrafficSignsDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.classes = os.listdir(data_path)
        self.images = []
        self.labels = []
        self.transform = transform
        self.load_data()

    def load_data(self):
        for class_id, class_name in enumerate(self.classes):
            class_folder = os.path.join(self.data_path, class_name)
            if not os.path.isdir(class_folder):
                continue
            for image_name in os.listdir(class_folder):
                if image_name.startswith('.'):
                    continue
                image_path = os.path.join(class_folder, image_name)
                img = cv2.imread(image_path)
                if img is not None:
                    img = cv2.resize(img, IMG_DIM)
                    self.images.append(img)
                    self.labels.append(class_id)

        self.images = np.array(self.images)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

In [4]:
class TrafficSignModel(nn.Module):
    def __init__(self, n_classes, n_kernels=60, n_nodes=500):
        super(TrafficSignModel, self).__init__()

        # Define kernel sizes and pool size
        kernel_size_5x5 = (5, 5)
        kernel_size_3x3 = (3, 3)
        pool_size = (2, 2)
        
        # First convolutional block
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_kernels, kernel_size=kernel_size_5x5)
        self.conv2 = nn.Conv2d(in_channels=n_kernels, out_channels=n_kernels, kernel_size=kernel_size_5x5)
        self.pool = nn.MaxPool2d(kernel_size=pool_size)

        # Second convolutional block
        self.conv3 = nn.Conv2d(in_channels=n_kernels, out_channels=n_kernels // 2, kernel_size=kernel_size_3x3)
        self.conv4 = nn.Conv2d(in_channels=n_kernels // 2, out_channels=n_kernels // 2, kernel_size=kernel_size_3x3)

        # Dropout layer
        self.dropout = nn.Dropout(0.5)

        # Fully connected layers
        conv_output_size = self._get_conv_output_size()
        self.fc1 = nn.Linear(conv_output_size, n_nodes)
        self.fc2 = nn.Linear(n_nodes, n_classes)

    def _get_conv_output_size(self):
        # Pass a dummy input through the conv layers to calculate the output size
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, IMG_DIM[0], IMG_DIM[1])
            x = nn.functional.relu(self.conv1(dummy_input))
            x = nn.functional.relu(self.conv2(x))
            x = self.pool(x)

            x = nn.functional.relu(self.conv3(x))
            x = nn.functional.relu(self.conv4(x))
            x = self.pool(x)

            output_size = x.numel()  # Flatten size
        return output_size

    def forward(self, x):
        # First conv block
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)

        # Second conv block
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout(x)

        # Flatten the output
        x = torch.flatten(x, 1)

        # Fully connected layers
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        # Output with softmax activation
        return nn.functional.softmax(x, dim=1)

In [5]:
# Load the dataset
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize grayscale images
])

dataset = TrafficSignsDataset(DATA_PATH, transform=transform)
n_classes = len(dataset.classes)

In [8]:
# Split dataset into train, val, and test sets
test_size = int(TEST_SPLIT * len(dataset))
val_size = int(VAL_SPLIT * (len(dataset) - test_size))
train_size = len(dataset) - test_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
# Initialize model, loss function, and optimizer
model = TrafficSignModel(n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [68]:
# Training loop
for epoch in range(N_EPOCHS):
    model.train()
    running_loss = 0.0
    correct_train = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct_train += (outputs.argmax(dim=1) == labels).sum().item()

    # Validation
    model.eval()
    correct_val = 0
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()
            correct_val += (outputs.argmax(dim=1) == labels).sum().item()

    # Print epoch results
    print(f"Epoch {epoch + 1}/{N_EPOCHS}, "
          f"Train Loss: {running_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {correct_train / len(train_loader.dataset):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, "
          f"Val Accuracy: {correct_val / len(val_loader.dataset):.4f}")

Epoch 1/30, Train Loss: 3.4324, Train Accuracy: 0.2495, Val Loss: 3.3438, Val Accuracy: 0.3390
Epoch 2/30, Train Loss: 3.3308, Train Accuracy: 0.3512, Val Loss: 3.2801, Val Accuracy: 0.4015
Epoch 3/30, Train Loss: 3.2892, Train Accuracy: 0.3926, Val Loss: 3.2630, Val Accuracy: 0.4190
Epoch 4/30, Train Loss: 3.2731, Train Accuracy: 0.4087, Val Loss: 3.2387, Val Accuracy: 0.4434
Epoch 5/30, Train Loss: 3.2511, Train Accuracy: 0.4310, Val Loss: 3.2323, Val Accuracy: 0.4489
Epoch 6/30, Train Loss: 3.2356, Train Accuracy: 0.4463, Val Loss: 3.2078, Val Accuracy: 0.4738
Epoch 7/30, Train Loss: 3.2258, Train Accuracy: 0.4559, Val Loss: 3.2082, Val Accuracy: 0.4739
Epoch 8/30, Train Loss: 3.2193, Train Accuracy: 0.4625, Val Loss: 3.2051, Val Accuracy: 0.4765
Epoch 9/30, Train Loss: 3.2145, Train Accuracy: 0.4671, Val Loss: 3.1940, Val Accuracy: 0.4878
Epoch 10/30, Train Loss: 3.2069, Train Accuracy: 0.4745, Val Loss: 3.1775, Val Accuracy: 0.5046
Epoch 11/30, Train Loss: 3.1956, Train Accuracy: 

In [71]:
# Testing
model.eval()
correct_test = 0
test_loss = 0.0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        test_loss += criterion(outputs, labels).item()
        correct_test += (outputs.argmax(dim=1) == labels).sum().item()

print(f"Test Loss: {test_loss / len(test_loader):.4f}, "
      f"Test Accuracy: {correct_test / len(test_loader.dataset):.4f}")

# save the trained model
pickle_out = open(RESULT_PATH, "wb")
pickle.dump(model, pickle_out) # serialize and save the model
pickle_out.close()

Test Loss: 3.1399, Test Accuracy: 0.5419
